## Clean our data

In [15]:
from bs4 import BeautifulSoup as bs
import requests



In [16]:
import json

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [17]:
movie_info_list=load_data("academy_awards.json")

#### tasks
- Convert running time into an integer
- Convert dates into datetime object
- Split up the long strings
- Convert money to number


In [18]:
movie_info_list[100]

{'title': 'Rebecca',
 'Directed by': 'Alfred Hitchcock',
 'Screenplay by': ['Robert E. Sherwood',
  'Joan Harrison',
  'Adaptation:',
  'Philip MacDonald',
  'Michael Hogan'],
 'Based on': ['Rebecca', 'by', 'Daphne du Maurier'],
 'Produced by': 'David O. Selznick',
 'Starring': ['Laurence Olivier', 'Joan Fontaine'],
 'Cinematography': 'George Barnes',
 'Edited by': 'W. Donn Hayes',
 'Music by': 'Franz Waxman',
 'Production company': 'Selznick International Pictures',
 'Distributed by': 'United Artists',
 'Release date': ['March 21, 1940 ( Miami, Florida )', 'April 12, 1940 (US)'],
 'Running time': '130 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$1.29 million',
 'Box office': '$6 million'}

#### Converting running time into an integer

In [19]:
def minutes_to_integer(running_time):
    if running_time == "N/A":
        return None
    
    if isinstance(running_time, list):
        if running_time[0].split(" ")[0]=="Original":
            return running_time[1].split(" ")[0]
        else:
            return running_time[0].split(" ")[0]
    else: # is a string
        try:
            return int(running_time.split(" ")[0])
        except:
            return None 

for movie in movie_info_list:
    movie['Running time (int)'] = minutes_to_integer(movie.get('Running time', "N/A"))
#Other more difficult edge cases I will fix with pandas

In [20]:
print([movie.get('Running time (int)', 'N/A') for movie in movie_info_list])


['111', 110, 84, 100, 90, '130', 95, 113, '152', 87, '90', 84, 107, 124, 102, 101, 85, 122, 112, 108, 90, 87, 89, 80, 80, 89, 112, 89, 88, 93, 96, 115, 97, 66, 98, 97, 105, 110, 100, 97, 107, 87, 88, 111, 83, 91, 115, 80, 132, 99, 101, 119, 129, 91, 109, '133', 103, 103, 90, 101, '177', 141, 101, 98, 116, 125, 115, 87, 123, 84, 116, 91, 115, 93, 138, '96', 132, 85, 92, 111, 126, 102, 106, 93, 110, 90, 114, 103, 96, 118, '221', 104, 114, 87, None, 110, 107, 96, 101, 103, 130, 141, 120, 129, 124, 108, 95, 105, 90, 112, 118, 99, 119, 94, 116, 115, 101, 108, 134, 99, 133, 123, 127, '88', 87, 128, 125, 117, 88, 126, 102, 170, 112, 118, 115, 124, 104, 75, 155, 114, 126, 107, 114, 172, 154, 101, 143, 126, 111, 111, 172, 136, 131, 145, 128, 118, 109, 86, 118, 96, 155, 102, 133, 108, 126, 110, 118, 115, 103, 132, 138, 102, 92, 103, 110, 113, 119, 122, 171, 125, 152, 85, 107, 119, 129, 118, 121, 135, 118, 118, 108, 125, 104, 102, 102, 90, 102, '120,', 115, 117, 182, 137, 197, '133', '220', 161, 

In [21]:
#Convert budget and box office money into integer
print([movie.get('Budget', 'N/A') for movie in movie_info_list])

['US$ 2 million ($28,850,173 adjusted for inflation)', '$1.3 million', 'N/A', '$379,000', 'N/A', '$426,000', 'N/A', '$1 million', '$1.2 million', '$414,000', '$318,000', 'N/A', 'N/A', '$1,433,000', 'N/A', 'N/A', 'N/A', '$1.3 million', '$750,000', 'N/A', 'Under $100,000', '$356,000', '$310,000', 'N/A', 'N/A', 'N/A', '$1,180,280', '$439,000', '$900,000', '$228,000', '$300,000', '$424,000', '£65,000', '$200,000', '$851,000', '$600,000', '$325,000', '$820,000', '$842,908', '$479,000', '$520,000', '$263,000', 'N/A', 'N/A', '$733,150', '$226,408', '$1,022,000', 'N/A', '$1,950,000', '$342,000', '$1,062,000', '$1,242,000 or $995,000', '$1,073,000', '$243,000', 'N/A', '$981,000', 'N/A', '$782,000', 'N/A', '$ 609,000', '$2.183 million', '$1,192,000', 'N/A', '$603,000', '$845,710', '$2 million', '$1.3 million', 'N/A', '$1,232,000', '$326,000 or $319,107', 'N/A', '$600,000 ($10,800,000 in 2020 dollars)', '$1,645,000', '$900,000 (est)', '$2,816,000', 'N/A', '$2 million', '$762,000 or $733,000', '$9

In [22]:
#len([movie.get('Budget', 'N/A') for movie in movie_info_list if movie.get('Budget', 'N/A')=='N/A'])

print([movie.get('Budget', 'N/A') for movie in movie_info_list].count('N/A'))



58


In [23]:
print([movie.get('Box office', 'N/A') for movie in movie_info_list].count('N/A'))

29


#### Budget and Box office money conversion 

In [24]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re = rf"\${number}"

def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict[word]

def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    word = re.search(amounts, string, flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    return value

'''
money_conversion("$12.2 million") --> 12200000 ## Word syntax
money_conversion("$790,000") --> 790000        ## Value syntax
'''
def money_conversion(money):
    if money == "N/A":
        return None

    if isinstance(money, list):
        money = money[0]
        
    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())

    else:
        return None

In [25]:
for movie in movie_info_list:
    movie['Budget (float)'] = money_conversion(movie.get('Budget', "N/A"))
    movie['Box office (float)'] = money_conversion(movie.get('Box office', "N/A"))

In [26]:
money_conversion(str(movie_info_list[-40]["Budget"]))

40000000.0

### Convert Dates into datetimes

In [27]:
# Convert Dates into datetimes
print([movie.get('Release date', 'N/A') for movie in movie_info_list])


[['August 12, 1927 (New York City, premiere)', 'January 15, 1928 (Los Angeles)', 'January 5, 1929 (United States)'], ['May 6, 1927 (Los Angeles)', 'May 25, 1927 (New York City)', 'September 10, 1927 (New York City (re-release))'], ['November 1, 1928'], ["February 1, 1929 ( Grauman's Chinese Theatre )", 'February 8, 1929 ( NYC )', 'June 12, 1929 ( US )'], ['April 20, 1929'], ['June 20, 1929'], ['Premiere:', 'December\xa025,\xa01928', 'General release:', 'January\xa020,\xa01929'], 'August 17, 1928', ['April 21, 1930 (US )'], 'June 24, 1930', ['November 1, 1929'], ['April 30, 1930 (United States)'], ['November 19, 1929', '(New York City)', 'January 18, 1930', '(US)'], ['January 26, 1931 (Premiere- New York City )', 'February 9, 1931 ( US )'], ['February 20, 1931 (New York City)', 'March 31, 1931 (U.S.)'], ['April 4, 1931'], ['April 5, 1931'], ['May 23, 1931 (United States)'], ['April 12, 1932 ( US )'], 'December 7, 1931', 'September 19, 1931 (US)', ['November 9, 1931'], ['September 26, 19

In [28]:
from datetime import datetime

dates = [movie.get('Release date', 'N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
        
    if date == "N/A":
        return None
        
    date_str = clean_date(date)

    fmts = ["%B %d, %Y", "%d %B %Y","%B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None


In [29]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

In [30]:
movie_info_list[50]

{'title': 'Broadway Melody of 1936',
 'Directed by': 'Roy Del Ruth',
 'Written by': ['Harry W. Conn', 'Moss Hart', 'Jack McGowan', 'Sid Silvers'],
 'Produced by': 'John W. Considine Jr.',
 'Starring': ['Jack Benny',
  'Eleanor Powell',
  'Robert Taylor',
  'Una Merkel',
  'Frances Langford',
  'Sid Silvers',
  'Buddy Ebsen',
  'Vilma Ebsen'],
 'Cinematography': 'Charles Rosher',
 'Edited by': 'Blanche Sewell',
 'Music by': 'Nacio Herb Brown',
 'Distributed by': 'Metro-Goldwyn-Mayer',
 'Release date': ['August 25, 1935 ( New York City )',
  'September 18, 1935 ( Los Angeles )'],
 'Running time': '101 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$1,062,000',
 'Box office': '$2,871,000',
 'Running time (int)': 101,
 'Budget (float)': 1062000.0,
 'Box office (float)': 2871000.0,
 'Release date (datetime)': datetime.datetime(1935, 8, 25, 0, 0)}

In [33]:
#checking the number of datetime.datetime objects
import collections
a=[movie.get('Release date (datetime)', 'N/A') for movie in movie_info_list]
map_object = map(type, a)
new_list = list(map_object)
occurrences = collections.Counter(new_list)
print(occurrences)

Counter({<class 'datetime.datetime'>: 569, <class 'NoneType'>: 2})


In [48]:
def year_conversion(datetim):
    if isinstance(datetim, datetime):
        return datetim.year
    else:
        return None
        
#print([if movie.get('Release date (datetime)', 'N/A').year for movie in movie_info_list])

In [49]:
for movie in movie_info_list:
    movie['YEAR'] = year_conversion(movie.get('Release date (datetime)', 'N/A'))

In [52]:
print(len([movie.get('YEAR', 'N/A') for movie in movie_info_list]))
#def save_data(title, data):
#    with open(title, 'w', encoding='utf-8') as f:
#        json.dump(data, f, ensure_ascii=False, indent=2)
#save_data("academy_international_movies_cleane.json", movie_info_list)

571


#### Because Object of type datetime is not JSON serializable I will use a pickle to save the data

In [53]:
import pickle

def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

In [54]:
import pickle

def load_data_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [55]:
save_data_pickle("academy_movies_clean.pickle", movie_info_list)

In [56]:
a = load_data_pickle("academy_movies_clean.pickle")

In [57]:
a == movie_info_list

True

### Attach IMDB/Rotten Tomatoes/Metascore scores 

In [59]:
movie_info_list = load_data_pickle('academy_movies_clean.pickle')

The OMDb API is a RESTful web service to obtain movie information, all content and images on the site are contributed and maintained by our users.
 

In [ ]:
#http://www.omdbapi.com/?apikey=[yourkey]&

In [66]:
import requests
import urllib
import os
import time

def get_omdb_info(title,YEAR):
    base_url = "http://www.omdbapi.com/?"
    parameters = {"apikey": "56c6630c", 't': title, 'y': YEAR}
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(omdb_info):
    ratings = omdb_info.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None

get_omdb_info("Rebecca", 1940)

{'Title': 'Rebecca',
 'Year': '1940',
 'Rated': 'Approved',
 'Released': '12 Apr 1940',
 'Runtime': '130 min',
 'Genre': 'Drama, Mystery, Romance',
 'Director': 'Alfred Hitchcock',
 'Writer': 'Daphne Du Maurier, Robert E. Sherwood, Joan Harrison',
 'Actors': 'Laurence Olivier, Joan Fontaine, George Sanders',
 'Plot': "A self-conscious woman juggles adjusting to her new role as an aristocrat's wife and avoiding being intimidated by his first wife's spectral presence.",
 'Language': 'English, French',
 'Country': 'United States',
 'Awards': 'Won 2 Oscars. 7 wins & 10 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BYTcxYWExOTMtMWFmYy00ZjgzLWI0YjktNWEzYzJkZTg0NDdmL2ltYWdlXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.1/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '100%'},
  {'Source': 'Metacritic', 'Value': '86/100'}],
 'Metascore': '86',
 'imdbRating': '8.1',
 'imdbVotes': '131,280',
 'imdbID': 'tt00329

In [67]:
for movie in movie_info_list:
    title = movie['title']
    YEAR = movie['YEAR']
    time.sleep(2)
    omdb_info = get_omdb_info(title,YEAR)    
    movie['imdb'] = omdb_info.get('imdbRating', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)
    movie['plot'] =  omdb_info.get('Plot', None)
    movie['genre'] =  omdb_info.get('Genre', None)
    movie['awards'] =  omdb_info.get('Awards', None)

In [71]:
movie_info_list[569]

{'title': 'Sound of Metal',
 'Directed by': 'Darius Marder',
 'Screenplay by': ['Darius Marder', 'Abraham Marder'],
 'Story by': ['Darius Marder', 'Derek Cianfrance'],
 'Produced by': ['Bill Benz',
  'Kathy Benz',
  'Bert Hamelinck',
  'Sacha Ben Harroche'],
 'Starring': ['Riz Ahmed', 'Olivia Cooke', 'Paul Raci'],
 'Cinematography': 'Daniël Bouquet',
 'Edited by': 'Mikkel E. G. Nielsen',
 'Music by': ['Abraham Marder', 'Nicolas Becker'],
 'Production companies': ['Caviar', 'Ward Four', 'Flat 7 Productions'],
 'Distributed by': 'Amazon Studios',
 'Release date': ['September 6, 2019 ( TIFF )',
  'November 20, 2020 (United States)'],
 'Running time': '120 minutes',
 'Country': 'United States',
 'Languages': ['American Sign Language', 'English'],
 'Budget': '$5.4 million',
 'Box office': '$514,343',
 'Running time (int)': 120,
 'Budget (float)': 5400000.0,
 'Box office (float)': 514343.0,
 'Release date (datetime)': datetime.datetime(2019, 9, 6, 0, 0),
 'YEAR': 2019,
 'imdb': '7.8',
 'meta

In [72]:
save_data_pickle('academy_movies_final.pickle', movie_info_list)

### Save data as JSON and CSV

In [73]:
movie_info_copy = [movie.copy() for movie in movie_info_list]

In [74]:
for movie in movie_info_copy:
    current_date = movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None

In [76]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [77]:
save_data("academy_movies_final.json", movie_info_copy)

Save data as CSV

In [78]:
import pandas as pd

df = pd.DataFrame(movie_info_list)

In [79]:
df.tail()

,title,Directed by,Written by,Screenplay by,Story by,Produced by,Starring,Cinematography,Edited by,Music by,...,Countries,Narrated by,Suggested by,Narration by,Traditional,Simplified,Mandarin,Hangul,Revised Romanization,McCune–Reischauer
566,Mank,David Fincher,NaN,Jack Fincher,NaN,"[Ceán Chaffin, Eric Roth, Douglas Urbanski]","[Gary Oldman, Amanda Seyfried, Lily Collins, A...",Erik Messerschmidt,Kirk Baxter,"[Trent Reznor, Atticus Ross]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
567,Minari,Lee Isaac Chung,Lee Isaac Chung,NaN,NaN,"[Dede Gardner, Jeremy Kleiner, Christina Oh]","[Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate C...",Lachlan Milne,Harry Yoon,Emile Mosseri,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
568,Promising Young Woman,Emerald Fennell,Emerald Fennell,NaN,NaN,"[Margot Robbie, Josey McNamara, Tom Ackerley, ...","[Carey Mulligan, Bo Burnham, Alison Brie, Clan...",Benjamin Kračun,Frédéric Thoraval,Anthony Willis,...,"[United Kingdom, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569,Sound of Metal,Darius Marder,NaN,"[Darius Marder, Abraham Marder]","[Darius Marder, Derek Cianfrance]","[Bill Benz, Kathy Benz, Bert Hamelinck, Sacha ...","[Riz Ahmed, Olivia Cooke, Paul Raci]",Daniël Bouquet,Mikkel E. G. Nielsen,"[Abraham Marder, Nicolas Becker]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,The Trial of the Chicago 7,Aaron Sorkin,Aaron Sorkin,NaN,NaN,"[Stuart M. Besser, Matt Jackson, Marc Platt, T...","[Yahya Abdul-Mateen II, Sacha Baron Cohen, Dan...",Phedon Papamichael,Alan Baumgarten,Daniel Pemberton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df.to_csv("academy_movies_final.csv")